## makemore: part 5

In [ ]:
# we skip lecture 4 on backpropagation !
# now we want: 1)to enlarge the context length AND ''fuse information in a hierarchical manner''
# 2) 'torchify' the code.......
# For (1) we use the approach in https://arxiv.org/abs/1609.03499...


In [ ]:
# The importance of embedding

# word2vec: skip-gram https://arxiv.org/pdf/1301.3781v3.pdf
# node2vec: https://arxiv.org/pdf/1607.00653.pdf


In [ ]:
import random
import torch
import torch.nn.functional as F
import torch.nn as nn
import matplotlib.pyplot as plt # for making figures
%matplotlib inline

In [ ]:
# read in all the words
random.seed(158)
words = open("data/nomi_italiani.txt", "r").read().splitlines()
random.shuffle(words)
print(len(words))
words[0:8]


In [ ]:
# build the vocabulary of characters and mappings to/from integers
chars = sorted(list(set("".join(words))))
stoi = {s: i + 1 for i, s in enumerate(chars)}
stoi["."] = 0
itos = {i: s for s, i in stoi.items()}
vocab_size = len(itos)
print(itos)
print(vocab_size)


In [ ]:
# build the dataset
block_size = 3  # 8 context length: how many characters do we take to predict the next one?...start with 3 !!


def build_dataset(words):
    X, Y = [], []

    for w in words:
        context = [0] * block_size
        for ch in w + ".":
            ix = stoi[ch]
            X.append(context)
            Y.append(ix)
            context = context[1:] + [ix]  # crop and append

    X = torch.tensor(X)
    Y = torch.tensor(Y)
    print(X.shape, Y.shape)
    return X, Y


n1 = int(0.8 * len(words))
n2 = int(0.9 * len(words))
Xtr, Ytr = build_dataset(words[:n1])  # 80%
Xdev, Ydev = build_dataset(words[n1:n2])  # 10%
Xte, Yte = build_dataset(words[n2:])  # 10%


In [ ]:
for x, y in zip(Xtr[:20], Ytr[:20]):
    print("".join(itos[ix.item()] for ix in x), "-->", itos[y.item()])


In [ ]:
# Near copy paste of the layers we have developed in Part 3
# Comment on this ! .... https://pytorch.org/docs/stable/nn.html
# now just evaluate this then we will chage it and use torch functions !! comments in class....


# -----------------------------------------------------------------------------------------------
class Linear:
    def __init__(self, fan_in, fan_out, bias=True):
        self.weight = (
            torch.randn((fan_in, fan_out)) / fan_in**0.5
        )  # note: kaiming init
        self.bias = torch.zeros(fan_out) if bias else None

    def __call__(self, x):
        self.out = x @ self.weight
        if self.bias is not None:
            self.out += self.bias
        return self.out

    def parameters(self):
        return [self.weight] + ([] if self.bias is None else [self.bias])


# -----------------------------------------------------------------------------------------------
class BatchNorm1d:
    def __init__(self, dim, eps=1e-5, momentum=0.1):
        self.eps = eps
        self.momentum = momentum
        self.training = True
        # parameters (trained with backprop)
        self.gamma = torch.ones(dim)
        self.beta = torch.zeros(dim)
        # buffers (trained with a running 'momentum update')
        self.running_mean = torch.zeros(dim)
        self.running_var = torch.ones(dim)

    def __call__(self, x):
        # calculate the forward pass
        if self.training:
            if x.ndim == 2:
                dim = 0
            elif x.ndim == 3:
                dim = (0, 1)
            xmean = x.mean(dim, keepdim=True)  # batch mean
            xvar = x.var(dim, keepdim=True)  # batch variance
        else:
            xmean = self.running_mean
            xvar = self.running_var
        xhat = (x - xmean) / torch.sqrt(xvar + self.eps)  # normalize to unit variance
        self.out = self.gamma * xhat + self.beta
        # update the buffers
        if self.training:
            with torch.no_grad():
                self.running_mean = (
                    1 - self.momentum
                ) * self.running_mean + self.momentum * xmean
                self.running_var = (
                    1 - self.momentum
                ) * self.running_var + self.momentum * xvar
        return self.out

    def parameters(self):
        return [self.gamma, self.beta]


# -----------------------------------------------------------------------------------------------
class Tanh:
    def __call__(self, x):
        self.out = torch.tanh(x)
        return self.out

    def parameters(self):
        return []


# -----------------------------------------------------------------------------------------------
class Embedding:
    def __init__(self, num_embeddings, embedding_dim):
        self.weight = torch.randn((num_embeddings, embedding_dim))

    def __call__(self, IX):
        self.out = self.weight[IX]
        return self.out

    def parameters(self):
        return [self.weight]


# -----------------------------------------------------------------------------------------------
class Flatten:
    def __init__(self, n):
        self.n = n

    def __call__(self, x):
        self.out = x.view(x.shape[0], -1)
        return self.out

    def parameters(self):
        return []


# -----------------------------------------------------------------------------------------------
class FlattenConsecutive:
    def __init__(self, n):
        self.n = n

    def __call__(self, x):
        B, T, C = x.shape
        x = x.view(B, T // self.n, C * self.n)
        if x.shape[1] == 1:
            x = x.squeeze(1)
        self.out = x
        return self.out

    def parameters(self):
        return []


# -----------------------------------------------------------------------------------------------
class Sequential:
    def __init__(self, layers):
        self.layers = layers

    def __call__(self, x):
        for layer in self.layers:
            x = layer(x)
        self.out = x
        return self.out

    def parameters(self):
        # get parameters of all layers and stretch them out into one list
        return [p for layer in self.layers for p in layer.parameters()]


In [ ]:
torch.manual_seed(42)
# seed rng for reproducibility....just once


In [ ]:
# original network   chrome-extension://efaidnbmnnnibpcajpcglclefindmkaj/https://jmlr.org/papers/volume3/tmp/bengio03a.pdf

n_embd = 10  # the dimensionality of the character embedding vectors
n_hidden = 200  # the number of neurons in the hidden layer of the MLP

C = torch.randn(vocab_size, n_embd)
layers = [
    Linear(n_embd * block_size, n_hidden, bias=False),
    BatchNorm1d(n_hidden),
    Tanh(),
    Linear(n_hidden, vocab_size),
]

# parameter init


with torch.no_grad():
    layers[-1].weight *= 0.1  # last layer make less confident

parameters = [C] + [p for layer in layers for p in layer.parameters()]
print(sum(p.nelement() for p in parameters))  # number of parameters in total
for p in parameters:
    p.requires_grad = True


In [ ]:
# same optimization as last time
max_steps = 200000
batch_size = 32
lossi = []

for i in range(max_steps):
    # minibatch construct
    ix = torch.randint(0, Xtr.shape[0], (batch_size,))
    Xb, Yb = Xtr[ix], Ytr[ix]  # batch X,Y

    # forward pass

    emb = C[Xb]  # embed the characters into vectors
    x = emb.view(emb.shape[0], -1)  # concatenate the vectors
    for layer in layers:
        x = layer(x)
    loss = F.cross_entropy(x, Yb)  # loss function

    # backward pass
    for p in parameters:
        p.grad = None
    loss.backward()

    # update: simple SGD
    lr = 0.1 if i < 150000 else 0.01  # step learning rate decay
    for p in parameters:
        p.data += -lr * p.grad
        # track stats
    if i % 10000 == 0:  # print every once in a while
        print(f"{i:7d}/{max_steps:7d}: {loss.item():.4f}")
    lossi.append(loss.log10().item())


In [ ]:
# 32 batches are few...so you can get very lucky or unlucky....comment in class...

plt.plot(torch.tensor(lossi))


In [ ]:
torch.arange(12)  # .view(6,2)


In [ ]:
torch.tensor(lossi).view(-1, 1000)


In [ ]:
torch.tensor(lossi).view(-1, 1000).shape


In [ ]:
plt.plot(torch.tensor(lossi).view(-1, 1000).mean(1))  # mean on each row


In [ ]:
# put layers into eval mode (needed for batchnorm especially)...comment in class !
for layer in layers:
    layer.training = False


In [ ]:
# evaluate the loss
@torch.no_grad()  # this decorator disables gradient tracking inside pytorch
def split_loss(split):
    x, y = {
        "train": (Xtr, Ytr),
        "val": (Xdev, Ydev),
        "test": (Xte, Yte),
    }[split]
    emb = C[x]  # embed the characters into vectors (N,block_size)
    x = emb.view(emb.shape[0], -1)  # concatenate the vectors
    for layer in layers:
        x = layer(x)
    loss = F.cross_entropy(x, y)  # loss function
    print(split, loss.item())


split_loss("train")
split_loss("val")


In [ ]:
# sample from the model
for _ in range(20):
    out = []
    context = [0] * block_size  # initialize with all ...
    while True:
        # forward pass the neural net
        emb = C[
            torch.tensor([context])
        ]  # embed the characters into vectors (N,block_size)
        x = emb.view(emb.shape[0], -1)  # concatenate the vectors
        for layer in layers:
            x = layer(x)
        logits = x
        probs = F.softmax(logits, dim=1)
        # sample from the distribution
        ix = torch.multinomial(probs, num_samples=1).item()
        # shift the context window and track the samples
        context = context[1:] + [ix]
        out.append(ix)
        # if we sample the special '.' token, break
        if ix == 0:
            break

    print("".join(itos[i] for i in out))  # decode and print the generated word


In [ ]:
# we can do better and use "Embedding" (as pytorch) to see C as a first layer....


# original network   chrome-extension://efaidnbmnnnibpcajpcglclefindmkaj/https://jmlr.org/papers/volume3/tmp/bengio03a.pdf

n_embd = 10  # the dimensionality of the character embedding vectors
n_hidden = 200  # the number of neurons in the hidden layer of the MLP

# C= torch.randn(vocab_size,n_embd)
layers = [
    Embedding(vocab_size, n_embd),
    Flatten(block_size),
    Linear(n_embd * block_size, n_hidden, bias=False),
    BatchNorm1d(n_hidden),
    Tanh(),
    Linear(n_hidden, vocab_size),
]

# parameter init


with torch.no_grad():
    layers[-1].weight *= 0.1  # last layer make less confident

# parameters=[C]+[p for layer in layers for p in layer.parameters()]
parameters = [p for layer in layers for p in layer.parameters()]
print(sum(p.nelement() for p in parameters))  # number of parameters in total
for p in parameters:
    p.requires_grad = True


In [ ]:
# let us "torchizy" .... https://pytorch.org/docs/stable/nn.html

# discuss Sequential and Containers....


In [ ]:
# original network
# n_embd = 10 # the dimensionality of the character embedding vectors
# n_hidden = 200 # the number of neurons in the hidden layer of the MLP

# C= torch.randn(vocab_size,n_embd)
# layers=[Linear(n_embd*block_size,n_hidden,bias = False),BatchNorm1d(n_hidden),Tanh(),Linear(n_hidden, vocab_size),]

# original network "torchizied....""


n_embd = 10  # the dimensionality of the character embedding vectors
n_hidden = 200  # the number of neurons in the hidden layer of the MLP

model = nn.Sequential(
    nn.Embedding(vocab_size, n_embd),
    nn.Flatten(block_size),
    nn.Linear(n_embd * block_size, n_hidden, bias=False),
    nn.BatchNorm1d(n_hidden),
    nn.Tanh(),
    nn.Linear(n_hidden, vocab_size),
)

# parameter init
# with torch.no_grad()
#    layers=model.layers()
#    layers[-1].weight *= 0.1 # last layer make less confident # fix this !!

# parameters=[C]+[p for layer in layers for p in layer.parameters()]  before

print(sum(p.nelement() for p in model.parameters()))  # number of parameters in total
for p in model.parameters():
    p.requires_grad = True


In [ ]:
# or with 'karpathy' definitions....

n_embd = 10  # the dimensionality of the character embedding vectors
n_hidden = 200  # the number of neurons in the hidden layer of the MLP

model = Sequential(
    [
        Embedding(vocab_size, n_embd),
        Flatten(block_size),
        Linear(n_embd * block_size, n_hidden, bias=False),
        BatchNorm1d(n_hidden),
        Tanh(),
        Linear(n_hidden, vocab_size),
    ]
)

# parameter init
# with torch.no_grad()
#    layers=model.layers()
#    layers[-1].weight *= 0.1 # last layer make less confident # fix this !!

# parameters=[C]+[p for layer in layers for p in layer.parameters()]  before

print(sum(p.nelement() for p in model.parameters()))  # number of parameters in total
for p in model.parameters():
    p.requires_grad = True


In [ ]:
# put layers into eval mode (needed for batchnorm especially)
# be back on this now keep it !!

for layer in model.layers:
    layer.training = True

# model.train(True)


In [ ]:
# same optimization as before time
max_steps = 200000
batch_size = 32
lossi = []

for i in range(max_steps):
    # minibatch construct
    ix = torch.randint(0, Xtr.shape[0], (batch_size,))
    Xb, Yb = Xtr[ix], Ytr[ix]  # batch X,Y

    # forward pass
    #   emb=C[Xb] # embed the characters into vectors
    #   x=emb.view(emb.shape[0],-1) #concatenate the vectors
    #   for layer in layers:
    #     x=layer(x)

    logits = model(Xb)
    loss = F.cross_entropy(logits, Yb)  # loss function

    # backward pass
    for p in model.parameters():
        p.grad = None

    loss.backward()

    # update: simple SGD
    lr = 0.1 if i < 150000 else 0.01  # step learning rate decay
    for p in model.parameters():
        p.data += -lr * p.grad

    # track stats
    if i % 10000 == 0:  # print every once in a while
        print(f"{i:7d}/{max_steps:7d}: {loss.item():.4f}")
    lossi.append(loss.log10().item())
# break


In [ ]:
plt.plot(torch.tensor(lossi))


In [ ]:
plt.plot(torch.tensor(lossi).view(-1, 1000).mean(1))  # mean on each row


In [ ]:
# put layers into eval mode (needed for batchnorm especially)
# be back on this now keep it !!

for layer in model.layers:
    layer.training = False

# model.train(True)


In [ ]:
# evaluate the loss
@torch.no_grad()  # this decorator disables gradient tracking inside pytorch
def split_loss(split):
    x, y = {
        "train": (Xtr, Ytr),
        "val": (Xdev, Ydev),
        "test": (Xte, Yte),
    }[split]

    #   emb=C[x] # embed the characters into vectors (N,block_size) before
    #   x=emb.view(emb.shape[0],-1) #concatenate the vectors

    #     for layer in layers:
    #         x=layer(x)

    logits = model(x)

    loss = F.cross_entropy(logits, y)  # loss function

    print(split, loss.item())


split_loss("train")
split_loss("val")


In [ ]:
# sample from the model
for _ in range(20):
    out = []
    context = [0] * block_size  # initialize with all ...
    while True:
        # forward pass the neural net
        #         emb=C[torch.tensor([context])] # embed the characters into vectors (N,block_size)
        #         x=emb.view(emb.shape[0],-1) #concatenate the vectors
        #         for layer in layers:
        #             x=layer(x)
        #         logits = x
        logits = model(torch.tensor([context]))
        probs = F.softmax(logits, dim=1)
        # sample from the distribution
        ix = torch.multinomial(probs, num_samples=1).item()
        # shift the context window and track the samples
        context = context[1:] + [ix]
        out.append(ix)
        # if we sample the special '.' token, break
        if ix == 0:
            break

    print("".join(itos[i] for i in out))  # decode and print the generated word


In [ ]:
# now we are going to try to do better...but because we crunch all the characters at the first  layer (show picture language model),
# adding deeper layer will not be useful...we need first to do as wavenet: cluster characters in steps, hierarchialy...(convolution)
# Progressive fusion along the layers: 2-gram -> 4->gram -> 8-gram....show picture in wavenet paper.....

# first recreate the data set with block_size = 8


In [ ]:
ix = torch.randint(0, Xtr.shape[0], (4,))  # let's look at a batch of just 4 example
Xb, Yb = Xtr[ix], Ytr[ix]
logits = model(Xb)
print(Xb.shape)
Xb


In [ ]:
model.layers[0].out.shape  # output of the embedding layer


In [ ]:
model.layers[1].out.shape  # output of the Flatten layer


In [ ]:
model.layers[2].out.shape  # output of the Linear layer


In [ ]:
# now a nice surprise about the Linear Layer https://pytorch.org/docs/stable/generated/torch.nn.Linear.html#torch.nn.Linear

# just to look at the dimensions

(
    torch.randn(4, 80) @ torch.randn(80, 200) + torch.randn(200)
).shape  # broadcasting in the last term....


In [ ]:
# but also !!
(torch.randn(4, 2, 80) @ torch.randn(80, 200) + torch.randn(200)).shape


In [ ]:
# or also !!
(torch.randn(4, 4, 20) @ torch.randn(20, 200) + torch.randn(200)).shape


In [ ]:
# 1 2 3 4 5 6 7 8 -> (1 2) (3 4) (5 6)(7 8)
# we want to fuse vectors in pairs and acts in parallel on the 4 pairs of characters.
# i.e. we go from (torch.randn(4,80)@ torch.randn(80,200)+torch.randn(200)).shape
# ....second batch dimension...discuss in class...!!

(torch.randn(4, 4, 20) @ torch.randn(20, 200) + torch.randn(200)).shape


In [ ]:
# so now we need to change the Flatten layer to produce a (4,4,20)tensor and NOT (4,80)
e = torch.randn(
    4, 8, 10
)  # goal: want this to be (4,4,20) where consecutive 10-d vectors
# get concatenated (in pairs (1 2) (3 4) (5 6) (7 8))


In [ ]:
# trick

print(list(range(10)))
print(list(range(10))[::2])
print(list(range(10))[1::2])


In [ ]:
# so we want this...
print(e.shape)
explicit = torch.cat([e[:, ::2, :], e[:, 1::2, :]], dim=2)
explicit.shape


In [ ]:
# but ALSO this works !!

(e.view(4, 4, 20) == explicit)  # .all()   try this...


In [ ]:
input = torch.randn(4, 8, 10)
print(input.shape)
m = nn.Flatten()
output = m(input)
output.shape


In [ ]:
# -----------------------------------------------------------------------------------------------
class FlattenConsecutive:
    def __init__(self, n):
        self.n = n

    def __call__(self, x):
        B, T, C = x.shape
        x = x.view(B, T // self.n, C * self.n)
        if x.shape[1] == 1:
            x = x.squeeze(1)
        self.out = x
        return self.out

    def parameters(self):
        return []


In [ ]:
# Back to the Model...here we recover the previous one with FlattenConsecutive(block_size)


n_embd = 10  # the dimensionality of the character embedding vectors
n_hidden = 200  # the number of neurons in the hidden layer of the MLP

# model = nn.Sequential(
#     nn.Embedding(vocab_size, n_embd),nn.Flatten(), nn.Linear(n_embd * block_size, n_hidden, bias=False),
#     nn.BatchNorm1d(n_hidden), nn.Tanh(),
#     nn.Linear(n_hidden, vocab_size))

model = Sequential(
    [
        Embedding(vocab_size, n_embd),
        FlattenConsecutive(block_size),
        Linear(n_embd * block_size, n_hidden, bias=False),
        BatchNorm1d(n_hidden),
        nn.Tanh(),
        Linear(n_hidden, vocab_size),
    ]
)


# parameter init


# with torch.no_grad()
#    layers=model.layers()
#    layers[-1].weight *= 0.1 # last layer make less confident # fix this !!

# parameters=[C]+[p for layer in layers for p in layer.parameters()]  before

print(sum(p.nelement() for p in model.parameters()))  # number of parameters in total
for p in model.parameters():
    p.requires_grad = True


In [ ]:
ix = torch.randint(0, Xtr.shape[0], (4,))  # let's look at a batch of just 4 example
Xb, Yb = Xtr[ix], Ytr[ix]
logits = model(Xb)
print(Xb.shape)
Xb


In [ ]:
for layer in model.layers[:-2]:
    print(layer.__class__.__name__, ":", tuple(layer.out.shape))


In [ ]:
# we now move to a hierarchical approach

n_embd = 10  # the dimensionality of the character embedding vectors
n_hidden = 200  # the number of neurons in the hidden layer of the MLP

model = Sequential(
    [
        Embedding(vocab_size, n_embd),
        FlattenConsecutive(2),
        Linear(n_embd * 2, n_hidden, bias=False),
        BatchNorm1d(n_hidden),
        Tanh(),
        FlattenConsecutive(2),
        Linear(n_hidden * 2, n_hidden, bias=False),
        BatchNorm1d(n_hidden),
        Tanh(),
        FlattenConsecutive(2),
        Linear(n_hidden * 2, n_hidden, bias=False),
        BatchNorm1d(n_hidden),
        Tanh(),
        Linear(n_hidden, vocab_size),
    ]
)


print(sum(p.nelement() for p in model.parameters()))  # number of parameters in total
for p in model.parameters():
    p.requires_grad = True


In [ ]:
ix = torch.randint(0, Xtr.shape[0], (4,))  # let's look at a batch of just 4 example
Xb, Yb = Xtr[ix], Ytr[ix]
logits = model(Xb)
print(Xb.shape)
Xb


In [ ]:
for layer in model.layers:
    print(layer.__class__.__name__, ":", tuple(layer.out.shape))


In [ ]:
logits.shape


In [ ]:
# we now move to a hierarchical approach but resize to compare with the initial onwe
# we use n_hidden=68 to have almost the same number of parameters..,

n_embd = 10  # the dimensionality of the character embedding vectors
n_hidden = 200  # the number of neurons in the hidden layer of the MLP

model = Sequential(
    [
        Embedding(vocab_size, n_embd),
        FlattenConsecutive(2),
        Linear(n_embd * 2, n_hidden, bias=False),
        BatchNorm1d(n_hidden),
        Tanh(),
        FlattenConsecutive(2),
        Linear(n_hidden * 2, n_hidden, bias=False),
        BatchNorm1d(n_hidden),
        Tanh(),
        FlattenConsecutive(2),
        Linear(n_hidden * 2, n_hidden, bias=False),
        BatchNorm1d(n_hidden),
        Tanh(),
        Linear(n_hidden, vocab_size),
    ]
)


print(sum(p.nelement() for p in model.parameters()))  # number of parameters in total
for p in model.parameters():
    p.requires_grad = True


In [ ]:
# same optimization as before
max_steps = 200000
batch_size = 32
lossi = []

for i in range(max_steps):
    # minibatch construct
    ix = torch.randint(0, Xtr.shape[0], (batch_size,))
    Xb, Yb = Xtr[ix], Ytr[ix]  # batch X,Y

    # forward pass

    logits = model(Xb)
    loss = F.cross_entropy(logits, Yb)  # loss function

    # backward pass

    for p in model.parameters():
        p.grad = None

    loss.backward()

    # update: simple SGD
    lr = 0.1 if i < 150000 else 0.01  # step learning rate decay
    for p in model.parameters():
        p.data += -lr * p.grad

    # track stats
    if i % 10000 == 0:  # print every once in a while
        print(f"{i:7d}/{max_steps:7d}: {loss.item():.4f}")
    lossi.append(loss.log10().item())
# break


In [ ]:
plt.plot(torch.tensor(lossi))


In [ ]:
plt.plot(torch.tensor(lossi).view(-1, 1000).mean(1))  # mean on each row


In [ ]:
# put layers into eval mode (needed for batchnorm especially)...comment in class !
for layer in model.layers:
    layer.training = False


In [ ]:
# evaluate the loss
@torch.no_grad()  # this decorator disables gradient tracking inside pytorch
def split_loss(split):
    x, y = {
        "train": (Xtr, Ytr),
        "val": (Xdev, Ydev),
        "test": (Xte, Yte),
    }[split]

    logits = model(x)

    loss = F.cross_entropy(logits, y)  # loss function

    print(split, loss.item())


split_loss("train")
split_loss("val")


In [ ]:
# sample from the model
for _ in range(20):
    out = []
    context = [0] * block_size  # initialize with all ...
    while True:
        logits = model(torch.tensor([context]))
        probs = F.softmax(logits, dim=1)
        # sample from the distribution
        ix = torch.multinomial(probs, num_samples=1).item()
        # shift the context window and track the samples
        context = context[1:] + [ix]
        out.append(ix)
        # if we sample the special '.' token, break
        if ix == 0:
            break

    print("".join(itos[i] for i in out))  # decode and print the generated word
